the classes will eventually be split up but they are put together for now to get their interactions correct <br>
sources: <br>
events: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-II/ <br>
portfolio: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-V/ <br>
execution handler: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-VI/

In [1]:
import numpy as np
import pandas as pd
import os

from abc import ABC, abstractmethod
from queue import PriorityQueue
from dataclasses import dataclass

In [2]:
quotes_filename = os.path.join("play_data", "XBTUSD_quotes_191214_0434.csv")
trades_filename = os.path.join("play_data", "XBTUSD_trades_191214_0434.csv")

In [ ]:
quotes_full = pd.read_csv(
    quotes_filename,
    index_col='recorded',
    parse_dates=['timestamp', 'recorded']
)

trades_full = pd.read_csv(
    trades_filename,
    index_col='received',
    parse_dates=['timestamp', 'received'],
    dtype={"side":"category"}
)

all_data = {
    "quotes": quotes_full,
    "trades": trades_full
}

display( quotes_full.head() )
display( trades_full.head() )

In [4]:
@dataclass
class FillEvent:
    __slots__ = "event_time", "symbol", "exchange", "quantity", "cost"
    event_time : pd.Timestamp
    symbol: str
    exchange: str
    quantity: int  # change in securities held
    cost: float  # change in cash held
        
    @property
    def commission(self):
        # will eventually contain exchange/broker-specific logic
        return self.cost * 0.005
    
    def __cmp__(self, other):
        return cmp(self.event_time, other.event_time)
    
@dataclass
class Signal:
    __slots__ = "symbol", "exchange", "signal_type", "strength"
    symbol: str
    exchange: str
    signal_type: str  # BUY/SELL/EXIT
    strength: float
    
@dataclass
class Order:
    __slots__ = "symbol", "exchange", "order_type", "direction", "quantity", "price"
    symbol: str
    exchange: str
    order_type: str  # MKT/LMT
    direction: str  # BUY/SELL
    quantity: int
    price: float  # only relevent to LMT

In [5]:
class Portfolio(ABC):
    @abstractmethod
    def update_fill(self):
        pass
    
    @abstractmethod
    def send_order_from_signal(self):
        pass
    
class BacktestPortfolio(Portfolio):
    def __init__(self, queue, start_time, symbol_list, all_data, initial_capital=10000.0):
        self.queue = queue  # event queue
        self.symbol_list = symbol_list 
        self.initial_capital = initial_capital
        self.current_time = start_time
        
        self.all_positions_list = [pd.Series(0, index=symbol_list, name=start_time)]  # list of series
        self.all_holdings_list = [self.construct_current_holdings(start_time)]  # list of series
        
        self.current_positions = pd.Series(0, index=symbol_list, name=start_time)
        self.current_holdings = self.construct_current_holdings(start_time)  # series

    @property
    def all_positions(self):
        return pd.DataFrame(self.all_positions_list)
    
    @property
    def all_holdings(self):
        return pd.DataFrame(self.all_holdings_list)
    
    def construct_current_holdings(self, time):  # market values of positions
        holdings = {symbol: 0.0 for symbol in symbol_list}
        holdings["capital"] = self.initial_capital
        holdings["commission"] = 0.0
        holdings["total"] = self.initial_capital
        return pd.Series(holdings, name=time)
    
    def update_fill(self, fill):
        self.current_time = fill.timestamp
        
        # maybe can split position and holding updates
        self.current_positions.name = fill.timestamp
        self.current_positions[fill.symbol] += fill.quantity
        self.all_positions_list.append(self.current_positions.copy())
        
        self.current_holdings.name = fill.timestamp
        self.current_holdings["capital"] -= fill.cost
        # assuming constant prices between txns
        self.current_holdings[fill.symbol] = self.current_positions[fill.symbol] * fill.cost
        self.current_holdings["commission"] += fill.commission
        
    def risk_check(self):
        return True
    
    def calculate_order_price(self, order_type):
        # relevant to limit orders
        if order_type == "MKT":
            return 0.0
        return 0.0
    
    def send_order_from_signal(self, sig):
        print(sig)
        
        order_type = "MKT"
        
        if sig.signal_type == "EXIT":
            if self.current_positions[signal.symbol] < 0:
                direction = "BUY"
            else:
                direction = "SELL"
        else:
            direction = sig.signal_type
            
        quantity = int(100 * sig.strength)
        price = self.calculate_order_price(order_type)
        
        if self.risk_check():
            self.generate_fill_from_order(
                Order(sig.symbol, sig.exchange, order_type, direction, quantity, price)
            )
    
    def calculate_fill_cost(self, order):
        fill_price = 7000  # will eventually use future data
        return order.quantity * fill_price

    def calculate_fill_quantity(self, order):
        return order.quantity  # assumes 100% fill

    def generate_fill_from_order(self, order):
        print(order)
        event_time = self.current_time + pd.Timedelta("3s")
        quantity = self.calculate_fill_quantity(order)
        cost = self.calculate_fill_cost(order)
        
        fill = FillEvent(event_time, order.symbol, order.exchange, quantity, cost)
        print(fill)
        self.queue.put((event_time, fill))
        event_time += pd.Timedelta("1s")
        fill = FillEvent(event_time, order.symbol, order.exchange, quantity, cost)
        print(fill)
        self.queue.put((event_time, fill))
    
symbol_list = "XBTUSD", "ETHUSD"
time0 = pd.Timestamp.utcnow() - pd.Timedelta("15s")
pq = PriorityQueue()

p = BacktestPortfolio(pq, time0, symbol_list, all_data)
s = Signal("XBTUSD", "BitMex", "BUY", 100.0)

p.send_order_from_signal(s)

display(pq.get())
display(pq.get())

Signal(symbol='XBTUSD', exchange='BitMex', signal_type='BUY', strength=100.0)
Order(symbol='XBTUSD', exchange='BitMex', order_type='MKT', direction='BUY', quantity=10000, price=0.0)
FillEvent(event_time=Timestamp('2020-01-12 03:25:59.546883+0000', tz='UTC'), symbol='XBTUSD', exchange='BitMex', quantity=10000, cost=70000000)
FillEvent(event_time=Timestamp('2020-01-12 03:26:00.546883+0000', tz='UTC'), symbol='XBTUSD', exchange='BitMex', quantity=10000, cost=70000000)


(Timestamp('2020-01-12 03:25:59.546883+0000', tz='UTC'),
 FillEvent(event_time=Timestamp('2020-01-12 03:25:59.546883+0000', tz='UTC'), symbol='XBTUSD', exchange='BitMex', quantity=10000, cost=70000000))

(Timestamp('2020-01-12 03:26:00.546883+0000', tz='UTC'),
 FillEvent(event_time=Timestamp('2020-01-12 03:26:00.546883+0000', tz='UTC'), symbol='XBTUSD', exchange='BitMex', quantity=10000, cost=70000000))